In [1]:
import tweepy
import json
import pandas as pd
from datetime import datetime
import nltk
import boto3
import s3fs
import os
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
# nltk.download('vader_lexicon') # Need this to run SIA()
# need to install pyarrow

In [9]:
with open("C:/Users/William Jiang/Documents/credentials.json") as f:
    d = json.load(f)
    cred_json=d["twitter_api"]
    s3_cred_json=d["s3-access"]
    

##authenticate to s3
s3 = boto3.client('s3',aws_access_key_id=s3_cred_json['ACCESS_KEY_ID'], 
                      aws_secret_access_key=s3_cred_json['SECRET_ACCESS_KEY'], 
                      region_name='ap-southeast-2')
response = s3.list_buckets()    
    
##auth using s3fs
fs = s3fs.S3FileSystem(anon=False, key=s3_cred_json['ACCESS_KEY_ID'], secret=s3_cred_json['SECRET_ACCESS_KEY'])

### Dumps in to outputtweriterbucket2 ###

In [86]:
####DUMPSHIT INTO outputttwiterbuckek2####
##### Just Clear the bucket manually before running########

# fs.rm('s3://outputtwitterbucket2/')

for file1 in fs.listdir('twitterdatalake'):
    category=file1['Key'].split('/')[-1]
    for file2 in fs.listdir(file1['Key']):
        file_name=file2['Key'].split('/')[-1]
        #Reset count Every file
        count=0
        for file3 in fs.listdir(file2['Key']):
            file_key=file3['Key']
            if ".csv" in file_key:
                #Concatenate the files within a filename

                with fs.open('s3://' + file_key, 'r',encoding="utf8") as f:
                    if count==0:
                        df=pd.read_csv(f, encoding='utf8')
#                         print('here')
                    else:
#                         print('hesdfsdf2')
                        df_merge=pd.read_csv(f, encoding='utf8')
                        df=df.append(df_merge)
                    count=count+1


        #Output to output bucket as consolidated csv's
        file_dir='s3://outputtwitterbucket2/'
        to_csv_timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')
        full_filename=file_dir+category +'/'+ file_name +'_'+  to_csv_timestamp
        with fs.open(full_filename+ '.csv','w', encoding="utf-8", newline = '\n') as f:
            df = df.replace('\n',' ', regex=True)
            df = df.replace('\r',' ', regex=True)
            df.to_csv(f, index=False)
        print("Done:" + full_filename)


Done:s3://outputtwitterbucket2/Australia/Adelaide_20200722_231640
Done:s3://outputtwitterbucket2/Australia/Brisbane_20200722_231646
Done:s3://outputtwitterbucket2/Australia/Canberra_20200722_231652
Done:s3://outputtwitterbucket2/Australia/GoldCoast_20200722_231659
Done:s3://outputtwitterbucket2/Australia/Melbourne_20200722_231705
Done:s3://outputtwitterbucket2/Australia/Perth_20200722_231711
Done:s3://outputtwitterbucket2/Australia/Sydney_20200722_231716


In [57]:
for file1 in fs.listdir('twitterdatalake'):
    folder_name=file1['Key'].split('/')[-1]
    print(folder_name)

Australia
Chess
EPL
Esports
Fashion
FastFood
NBA
Pets
Politics
Religion
RichList
Technology
Tennis
Universities
World


In [67]:
##New improved - does in a tree sort Algo for max

search_id_max=0
last_since_id=None
file_name='Cuba'
category='Politics'
for file1 in fs.listdir('twitterdatalake'):
    folder_name1=file1['Key'].split('/')[-1]
    if folder_name1==category:
        for file2 in fs.listdir(file1['Key']):
#             print(folder_name2)
            folder_name2=file2['Key'].split('/')[-1]
            if folder_name2==file_name:
                for file3 in fs.listdir(file2['Key']):
                    file_key=file3['Key']
                    if ".csv" in file_key:
                        print(file_key)
                        search_id=int(file_key.split('-')[1].split('.csv')[0])
                        if search_id>search_id_max:
                            search_id_max=search_id 

if search_id_max==0:
    last_since_id=None
else:
    last_since_id=search_id_max

print(last_since_id)

twitterdatalake/Politics/Cuba/20200719_111552-1284808854749032449.csv
twitterdatalake/Politics/Cuba/20200719_213400-1284964331025829889.csv
twitterdatalake/Politics/Cuba/20200720_080354-1285122770389528577.csv
twitterdatalake/Politics/Cuba/20200720_183405-1285281253256826887.csv
twitterdatalake/Politics/Cuba/20200721_045152-1285436708251738113.csv
twitterdatalake/Politics/Cuba/20200721_150408-1285590761631879169.csv
twitterdatalake/Politics/Cuba/20200722_011554-1285744844275687424.csv
1285744844275687424


In [63]:
search_id_list

[1284808854749032449,
 1284964331025829889,
 1285122770389528577,
 1285281253256826887,
 1285436708251738113,
 1285590761631879169,
 1285744844275687424]

In [73]:
with fs.open('s3://mykeysandstuff/query_list.csv', 'r') as f1:
    df_query_list=pd.read_csv(f1)

In [74]:
df_query_list

,file_name,group_name,category,search
0,Arsenal,Arsenal FC,EPL,#Arsenal OR #AFC OR #COYG
1,TottenhamHotspur,Tottenham Hotspur FC,EPL,#Tottenham OR #THFC OR #SPURS
2,ManchesterCity,Manchester City FC,EPL,#ManchesterCity OR #ManCity OR #MCFC
3,LeicesterCity,Leicester City FC,EPL,#LeicesterCity OR #LCFC
4,CrystalPalace,Crystal Palace FC,EPL,#CrystalPalace OR #CPFC
...,...,...,...,...
199,UniversityOfBritishColumbia,UniversityOfBritishColumbia,Universities,#UBC OR #UniversityOfBritishColumbia
200,UCSD,UCSD,Universities,#UCSD
201,ANU,ANU,Universities,#ANU
202,DeakinUniversity,DeakinUniversity,Universities,#DeakinUniversity


In [47]:
###Another wayu###
search_id_list=[]

response=s3.list_objects(Bucket='twitterdatalake')
for obj in response['Contents']:
    file_key=obj['Key']
    print(file_key)
    if ".csv" in file_key and 'Politics' in file_key and 'Cuba' in file_key:
        print(file_key)
        search_id=int(file_key.split('-')[1].split('.csv')[0])
        search_id_list.append(search_id)

Australia/Adelaide/20200714_125804-1283022480051892225.csv
Australia/Adelaide/20200714_160404-1283066566687547392.csv
Australia/Adelaide/20200714_185504-1283112460598292482.csv
Australia/Adelaide/20200714_214604-1283155653947613184.csv
Australia/Adelaide/20200715_003704-1283197887980150784.csv
Australia/Adelaide/20200715_032804-1283240147635470336.csv
Australia/Adelaide/20200715_061904-1283284412625371136.csv
Australia/Adelaide/20200715_091004-1283327735046139904.csv
Australia/Adelaide/20200715_120104-1283370019296927745.csv
Australia/Adelaide/20200715_120904-1283372415440097280.csv
Australia/Adelaide/20200716_030719-1283598486726823936.csv
Australia/Adelaide/20200716_172231-1283803472853426178.csv
Australia/Adelaide/20200717_171015-1284169507779612677.csv
Australia/Adelaide/20200718_043409-1284338682577616896.csv
Australia/Adelaide/20200718_130957-1284475167566004225.csv
Australia/Adelaide/20200718_215457-1284605230785519617.csv
Australia/Adelaide/20200719_064552-1284737742920974336.c

In [44]:
search_id_list

[]

In [41]:
max(search_id_list)

1285832292137603072

In [42]:
print(s3.list_buckets())

{'ResponseMetadata': {'RequestId': 'CE0246E71EA07F39', 'HostId': '4uBGVN5FCQQiVqboZ8hNzY+mlksOWQETzuhrldOxBZLrZC1NODQ5cu4463g/b5es+3GX+qse2+I=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '4uBGVN5FCQQiVqboZ8hNzY+mlksOWQETzuhrldOxBZLrZC1NODQ5cu4463g/b5es+3GX+qse2+I=', 'x-amz-request-id': 'CE0246E71EA07F39', 'date': 'Wed, 22 Jul 2020 11:27:38 GMT', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'Buckets': [{'Name': 'aws-athena-query-results-031220881404-ap-southeast-2', 'CreationDate': datetime.datetime(2020, 7, 20, 11, 19, 48, tzinfo=tzutc())}, {'Name': 'aws-glue-scripts-031220881404-ap-southeast-2', 'CreationDate': datetime.datetime(2020, 7, 15, 11, 31, 14, tzinfo=tzutc())}, {'Name': 'aws-glue-temporary-031220881404-ap-southeast-2', 'CreationDate': datetime.datetime(2020, 7, 15, 11, 31, 15, tzinfo=tzutc())}, {'Name': 'elasticbeanstalk-ap-southeast-2-031220881404', 'CreationDate': datetime.datetime(2020, 7, 14, 13